In [1]:
# generate sample_points

from csv import reader
from itertools import chain
from random import randint,random,seed
from skimage.io import imread

def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

def get_points(path_to_data, x_bounds=(-1.0, 1.0), y_bounds=(-1.0, 1.0)):
    points = []
    with open(path_to_data, newline='') as data_file:
        data = reader(data_file)
        for line in data:
            if line == '':
                continue
            l,x,y = line[:3]
            if l == '':
                continue
            else:
                x,y = float(x),float(y)
                if x_bounds[0] < x < x_bounds[1] and y_bounds[0] < y < y_bounds[1]:
                    points.append((x,y))
    return points

def gray_to_points(impath,threshold):
    image = imread(impath,as_gray=True)
    rows,cols = image.shape
    points = []
    for i in range(rows):
        for j in range(cols):
            if image[i,j] < threshold:
                points.append((i,j))
    print(rows*cols)
    return points

In [31]:
def quarter(points, rect):
    l,r,t,b = rect
    mid_x, mid_y = ((l+r)/2,(t+b)/2)
    if len(points) > 0:

        points_by_x = sorted(points, key=lambda p: p[0])
        nearest_x = min(points_by_x, key=lambda p: abs(mid_x - p[0]))
        mid_x_i = points_by_x.index(nearest_x)

        points_left_by_y = sorted(points_by_x[:mid_x_i], key=lambda p: p[1])
        if len(points_left_by_y) > 0:
            nearest_left_y = min(points_left_by_y, key=lambda p: abs(mid_y - p[1]))
            mid_left_y_i = points_left_by_y.index(nearest_left_y)
            points_left_top = points_left_by_y[:mid_left_y_i]
            points_left_bottom = points_left_by_y[mid_left_y_i:]
        else:
            points_left_top = []
            points_left_bottom = []

        points_right_by_y = sorted(points_by_x[mid_x_i:], key=lambda p: p[1])
        if len(points_right_by_y) > 0:
            nearest_right_y = min(points_right_by_y, key=lambda p: abs(mid_y - p[1]))
            mid_right_y_i = points_right_by_y.index(nearest_right_y)
            points_right_top = points_right_by_y[:mid_right_y_i]
            points_right_bottom = points_right_by_y[mid_right_y_i:]
        else:
            points_right_top = []
            points_right_bottom = []
    else:
        points_left_top = []
        points_left_bottom = []
        points_right_top = []
        points_right_bottom = []
    
    return [(points_left_top, [l, mid_x, t, mid_y]), 
            (points_right_top, [mid_x, r, t, mid_y]), 
            (points_right_bottom, [mid_x, r, mid_y, b]), 
            (points_left_bottom, [l, mid_x, mid_y, b])]

def build_pyramid(sample_points,bounds,depth):
    layers = [[(sample_points,bounds)]]
    for i in range(1,depth):
        tiles = flatten([quarter(points,rect) for points,rect in layers[i-1]])
        tiles.sort(key=lambda x: (x[1][0],x[1][2]))
        layers.append(tiles)
    return layers

In [3]:
from Tile import *

In [32]:
# 1. randomly sample N integer 2-tuples
# N = int(3000)
# minX = randint(-1e7,-1e2)
# maxX = randint(1e2,1e7)
# minY = randint(-1e7,-1e2)
# maxY = randint(1e2,1e7)
# sample_points = [(minX+(maxX*random()),minY+(maxY*random())) for _ in range(N)]

# 2. sample points from a thresholded image
# sample_points = gray_to_points('test.png', 0.5)

# 3. load points from a .csv
#### CHANGE ME #########################
path_to_data = '../data/roots_100.csv'
########################################
sample_points = get_points(path_to_data, x_bounds=(-1,1),y_bounds=(-1,1))
print(len(sample_points))

# establish bounding rectangle
left = min(p[0] for p in sample_points)
right = max(p[0] for p in sample_points)
set_width = right - left
top = min(p[1] for p in sample_points)
bottom = max(p[1] for p in sample_points)
set_height = bottom - top
print((left,right,top,bottom))
print(set_width)
print(set_height)

139717
(-0.9997379567, 0.9997379567, -0.9999939777, 0.9999939777)
1.9994759134
1.9999879554


In [33]:
# generate pyramid from sample_points
from math import log
d = 8
pyramid = build_pyramid(sample_points,[left,right,top,bottom],d)
layers = [len(l) for l in pyramid]
nb_tiles = sum(layers)
print(nb_tiles)
print(layers)
print([log(l)/log(4) for l in layers])

21845
[1, 4, 16, 64, 256, 1024, 4096, 16384]
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]


In [42]:
# generate Tile objects from pyramid
# TODO : make a Pyramid class to ensure each tile layer is sorted by the topleft rect coordinate
width = 100
tiles = [[] for _ in range(d)]
for z in range(d):
    for q in range(4**z):
        pointset, rect = pyramid[z][q]
        i = q % (2**z)
        j = q // (2**z)
        pos = (z,i,j)
        tile = Tile(len(tiles), pos, pointset, rect, width=width)
        tiles[z].append(tile)
    #print(z)
    #print('\n'.join([str((t.rect,t.pos)) for t in tiles[z]]))
    #print()
print([len(l) for l in tiles])

[1, 4, 16, 64, 256, 1024, 4096, 16384]


In [43]:
# run this cell to create a pyramid at PYR_DIR
###########
PYR_DIR = './zoomed_pyramid/'
###########
import os
if not os.path.isdir(PYR_DIR):
    os.mkdir(PYR_DIR)
assert len(tiles) > 0
for T in flatten(tiles):
    T.save(PYR_DIR)